In [5]:
from pprint import pprint
import re

_items = [
    {
        "model": "chatgpt-3.5",
        "url": "http://localhost:8080",
        "query": "What is sandblasting",
        "response": "Some ChatGPT Model response here...",
    },
    {
        "model": "claude-4",
        "url": "http://localhost:5454",
        "query": "What is bungee jumping",
        "response": "Some Claude Model response here...",
    },
]

_current_item = 0
_json = _items[_current_item]

_parameter = {"timeout": 45, "model": "chatgpt-4o"}

_now = "20251202_214522"

_input = {
    "item": _json,
    "all": lambda: _items,
    "first": lambda: _items[0],
    "last": lambda: _items[-1],
}

out_fields = {
    "retries": 10,
    "timeout": "{{$parameter.timeout}}",
    "timestamp": "{{$now}}.log",
    "model": "{{$json.url}}-extra",
    "url": "{{$input.item.url}}",
}


# -------------------------------------------
#  SAFE RESOLUTION OF {{ ... }} EXPRESSIONS
# -------------------------------------------


def resolve_single(expr_body, scope, errors, field_key):
    """Resolve $variable.attribute inside {{ }}."""
    if not expr_body.startswith("$"):
        errors.append(f"[{field_key}] Expression must start with $: {expr_body}")
        return None

    expr = expr_body[1:]  # remove leading $
    parts = expr.split(".")

    # Top-level reference
    if parts[0] not in scope:
        errors.append(f"[{field_key}] Unknown reference: {parts[0]}")
        return None

    obj = scope[parts[0]]

    # Resolve chained parts
    for p in parts[1:]:
        try:
            if p.endswith("()"):  # helper function call
                method = p[:-2]
                if isinstance(obj, dict) and method in obj and callable(obj[method]):
                    obj = obj[method]()
                else:
                    raise KeyError(f"'{method}' is not callable on {obj}")
            else:
                if isinstance(obj, dict) and p in obj:
                    obj = obj[p]
                else:
                    raise KeyError(f"Key '{p}' not found in {obj}")
        except Exception as e:
            errors.append(
                f"[{field_key}] Failed to resolve '{{{{{expr_body}}}}}': {str(e)}"
            )
            return None

    return obj


def resolve_string_value(value, scope, errors, field_key):
    """Resolve all {{ ... }} occurrences inside a string."""

    if not isinstance(value, str):
        return value  # literals untouched

    pattern = r"\{\{\s*(.*?)\s*\}\}"  # capture {{  ...  }}

    def repl(match):
        expr_body = match.group(1)
        resolved = resolve_single(expr_body, scope, errors, field_key)
        return "" if resolved is None else str(resolved)

    return re.sub(pattern, repl, value)


def resolve_dict(d, scope):
    resolved = {}
    errors = []

    for key, value in d.items():
        resolved[key] = resolve_string_value(value, scope, errors, key)

    return resolved, errors


# SCOPE AVAILABLE TO RESOLVER
scope = {"parameter": _parameter, "json": _json, "now": _now, "input": _input}

resolved_field, errors = resolve_dict(out_fields, scope)

print("Resolved:", resolved_field)
print("\nErrors:")
for err in errors:
    print(" -", err)

Resolved: {'retries': 10, 'timeout': '45', 'timestamp': '20251202_214522.log', 'model': 'http://localhost:8080-extra', 'url': 'http://localhost:8080'}

Errors:
